# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

<b> План работы: </b>
1. Загрузить данные, путь к файлу: /datasets/autos.csv.
2. Изучить данные. Заполнить пропущенные значения и обработать аномалии в столбцах. Если среди признаков имеются неинформативные, удалить их.
3. Подготовить выборки для обучения моделей.
4. Обучить разные модели, одна из которых — LightGBM, как минимум одна — не бустинг. Для каждой модели попробовать разные гиперпараметры.
5. Проанализировать время обучения, время предсказания и качество моделей.
6. Опираясь на критерии заказчика, выбрать лучшую модель, проверить её качество на тестовой выборке.

<b> Описание данных: </b>

<b> Признаки </b>
* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* Repaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя

<b> Целевой признак </b>

Price — цена (евро)

## Подготовка данных

In [1]:
!pip install pandas
!pip install numpy
!pip install skimpy
!pip install scikit-learn
!pip install lightgbm

In [2]:
import warnings
import time
import os

import pandas as pd
import numpy as np
from numpy.random import RandomState
from skimpy import clean_columns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor

warnings.filterwarnings('ignore')

STATE = RandomState(12345)

### Загрузка данных

In [3]:
try:
    data = pd.read_csv('autos.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv')

### Изучение данных

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Приведем название стобцов к стилю snake_case

In [6]:
data = clean_columns(data)
data.columns.tolist()

['date_crawled',
 'price',
 'vehicle_type',
 'registration_year',
 'gearbox',
 'power',
 'model',
 'kilometer',
 'registration_month',
 'fuel_type',
 'brand',
 'repaired',
 'date_created',
 'number_of_pictures',
 'postal_code',
 'last_seen']

Имеются следующие очевидно неинформативные признаки:
* date_crawled — дата скачивания анкеты из базы;
* registration_month — месяц регистрации автомобиля;
* date_created — дата создания анкеты;
* number_of_pictures — количество фотографий автомобиля;
* postal_code — почтовый индекс владельца анкеты (пользователя);
* last_seen — дата последней активности пользователя.

In [7]:
data = data.drop(['date_crawled', 'registration_month', 'date_created', 'number_of_pictures', 'postal_code', 'last_seen'], axis=1)

Проверим данные на явные дубликаты

In [8]:
data.duplicated().sum()

45040

Удалим 13372 дубликатов

In [9]:
data = data.drop_duplicates().reset_index(drop = True)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309329 entries, 0 to 309328
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              309329 non-null  int64 
 1   vehicle_type       274770 non-null  object
 2   registration_year  309329 non-null  int64 
 3   gearbox            292122 non-null  object
 4   power              309329 non-null  int64 
 5   model              290968 non-null  object
 6   kilometer          309329 non-null  int64 
 7   fuel_type          278565 non-null  object
 8   brand              309329 non-null  object
 9   repaired           244771 non-null  object
dtypes: int64(4), object(6)
memory usage: 23.6+ MB


In [11]:
data.describe()

,price,registration_year,power,kilometer
count,309329.000000,309329.000000,309329.000000,309329.000000
mean,4486.937196,2004.360105,110.976908,127217.735809
std,4564.852796,92.541399,200.969473,38532.941010
min,0.000000,1000.000000,0.000000,5000.000000
25%,1100.000000,1999.000000,69.000000,125000.000000
50%,2800.000000,2003.000000,105.000000,150000.000000
75%,6500.000000,2008.000000,143.000000,150000.000000
max,20000.000000,9999.000000,20000.000000,150000.000000


Проверим столбцы на пропуски

In [12]:
pd.DataFrame({'Количество пропусков': data.isna().sum(), 'Доля пропусков': data.isna().mean()*100}).rename_axis('Признаки', axis = 1)

Признаки,Количество пропусков,Доля пропусков
price,0,0.000000
vehicle_type,34559,11.172247
registration_year,0,0.000000
gearbox,17207,5.562686
power,0,0.000000
model,18361,5.935751
kilometer,0,0.000000
fuel_type,30764,9.945398
brand,0,0.000000
repaired,64558,20.870335


В столбце price (цена, евро) есть нулевые значения, удалим их

In [13]:
data = data[data['price'] != 0]

Посмотрим на уникальные значения в столбце vehicle_type (тип автомобильного кузова)

In [14]:
data['vehicle_type'].unique()

array([nan, 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

Заполним пропуски значением 'other'

In [15]:
data['vehicle_type'].fillna('other', inplace = True)

Посмотрим на уникальные значения в столбце registration_year (год регистрации автомобиля)

In [16]:
data['registration_year'].unique()

array([1993, 2011, 2004, 2001, 2008, 1995, 2014, 1998, 2005, 1910, 2016,
       2007, 2009, 2002, 2018, 1997, 1990, 2017, 1981, 2003, 1994, 1991,
       1984, 2006, 1999, 2012, 2010, 2000, 1992, 2013, 1996, 1985, 1989,
       2015, 1982, 1976, 1983, 1973, 1969, 1971, 1987, 1986, 1988, 1980,
       1970, 1965, 1945, 1925, 1974, 1979, 1955, 1978, 1972, 1968, 1977,
       1961, 1966, 1975, 1963, 1964, 1960, 5000, 1958, 1967, 1959, 1956,
       3200, 1000, 1941, 9999, 8888, 1500, 2200, 4100, 1962, 1929, 1957,
       1940, 3000, 2066, 1949, 2019, 1937, 1951, 1800, 1953, 1954, 1234,
       8000, 5300, 9000, 2900, 6000, 5900, 5911, 1400, 1950, 4000, 1948,
       1952, 8500, 1932, 1255, 3700, 3800, 4800, 1942, 7000, 1935, 1933,
       1936, 6500, 1923, 2290, 1930, 1001, 9450, 1944, 2500, 1943, 1934,
       1938, 1928, 5555, 5600, 1600, 1111, 2222, 1039, 1300, 2800, 1931,
       4500, 1602, 7800, 1947, 1927, 7100, 8200, 1946])

В столбце registration_year много аномалий. Удалим строки с годом меньше, чем 1960 и больше 2016, т.к. последняя анкета размещена в 2016 году.

In [17]:
data = data.loc[(data['registration_year'] >= 1960) & (data['registration_year'] <= 2016)]

Посмотрим на уникальные значения в столбце gearbox (тип коробки передач)

In [18]:
data['gearbox'].unique()

array(['manual', 'auto', nan], dtype=object)

Заполним пропуски значением 'unknown'

In [19]:
data['gearbox'].fillna('unknown', inplace = True)

Посмотрим на уникальные значения в столбце power (мощность л.с.)

In [20]:
data['power'].unique()

array([    0,   190,   163,    75,    69,   102,   109,   125,   101,
         105,   140,   115,   131,    60,   136,   160,   231,    50,
         118,   193,    99,   113,   218,   122,   129,    70,   306,
          95,    61,   177,   170,    55,   143,   286,   232,   150,
         156,    80,    82,    90,   155,    54,   185,    87,   180,
          86,    84,   224,   235,   200,   178,   265,    77,   110,
         144,   120,   116,   184,   126,   204,    88,   194,    64,
         305,   197,   179,   250,    45,   313,    41,   165,    98,
         130,   114,   211,    56,   201,   213,    58,   107,    83,
         174,   100,   220,    73,   192,    68,    66,   299,    74,
          52,   147,   310,    71,    97,    65,   239,   203,     5,
         300,   103,    85,   258,   320,    63,    81,   148,    44,
         145,   280,   260,   104,   188,   333,   186,   117,   141,
         132,   234,   158,    39,    92,    51,   135,    59,   230,
          53,   209,

Заменим нулевые значения на медианное значение конкретной модели и удалим записи с мощностью выше 1000 л.с.

In [21]:
data = data.loc[((data['power'] >= 50) & (data['power' ] <= 500) & (data['power'] != 0))]

Посмотрим на уникальные значения в столбце model (модель автомобиля)

In [22]:
data['model'].unique()

array([nan, 'grand', 'golf', 'fabia', '3er', '2_reihe', 'c_max',
       '3_reihe', 'passat', 'navara', 'polo', 'twingo', 'a_klasse',
       'scirocco', '5er', 'arosa', 'other', 'civic', 'transporter',
       'punto', 'e_klasse', 'clio', 'kadett', 'one', 'fortwo', '1er',
       'b_klasse', 'a8', 'jetta', 'fiesta', 'c_klasse', 'micra', 'vito',
       'sprinter', 'astra', '156', 'escort', 'forester', 'xc_reihe',
       'scenic', 'ka', 'a1', 'focus', 'a4', 'tt', 'a6', 'jazz', 'omega',
       'slk', '7er', 'combo', 'corsa', '80', '147', 'z_reihe', 'sorento',
       'ibiza', 'mustang', 'eos', 'touran', 'getz', 'insignia', 'almera',
       'megane', 'a3', 'r19', 'caddy', 'mondeo', 'cordoba', 'colt',
       'impreza', 'vectra', 'lupo', 'berlingo', 'tiguan', '6_reihe', 'c4',
       'panda', 'up', 'i_reihe', 'ceed', 'kangoo', '5_reihe', 'yeti',
       'octavia', 'zafira', 'mii', 'rx_reihe', '6er', 'modus', 'fox',
       'matiz', 'beetle', 'rio', 'touareg', 'logan', 'spider', 'cuore',
       's_m

Заполним пропуски значением 'unknown'

In [23]:
data['model'].fillna('unknown', inplace = True)

Посмотрим на уникальные значения в столбце kilometer (пробег, км)

In [24]:
data['kilometer'].unique()

array([125000, 150000,  90000,  30000,  70000, 100000,  60000,   5000,
        20000,  80000,  50000,  40000,  10000])

С данными в этом столбце все в порядке

Посмотрим на уникальные значения в столбце fuel_type (тип топлива)

In [25]:
data['fuel_type'].unique()

array(['gasoline', 'petrol', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

Заполним пропуски значением 'other'

In [26]:
data['fuel_type'].fillna('other', inplace = True)

Посмотрим на уникальные значения в столбце brand (марка автомобиля)

In [27]:
data['brand'].unique()

array(['audi', 'jeep', 'volkswagen', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'seat', 'honda',
       'fiat', 'opel', 'mini', 'smart', 'hyundai', 'sonstige_autos',
       'alfa_romeo', 'subaru', 'volvo', 'mitsubishi', 'kia', 'lancia',
       'citroen', 'toyota', 'chevrolet', 'dacia', 'suzuki', 'daihatsu',
       'chrysler', 'jaguar', 'daewoo', 'rover', 'porsche', 'saab',
       'land_rover', 'lada', 'trabant'], dtype=object)

С данными в этом столбце все в порядке

Посмотрим на уникальные значения в столбце repaired (была машина в ремонте или нет)

In [28]:
data['repaired'].unique()

array(['yes', nan, 'no'], dtype=object)

Заполним пропуски значением 'unknown'

In [29]:
data['repaired'].fillna('unknown', inplace = True)

In [30]:
pd.DataFrame({'Количество пропусков': data.isna().sum(), 'Доля пропусков': data.isna().mean()*100}).rename_axis('Признаки', axis = 1)

Признаки,Количество пропусков,Доля пропусков
price,0,0.0
vehicle_type,0,0.0
registration_year,0,0.0
gearbox,0,0.0
power,0,0.0
model,0,0.0
kilometer,0,0.0
fuel_type,0,0.0
brand,0,0.0
repaired,0,0.0


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 252490 entries, 1 to 309328
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              252490 non-null  int64 
 1   vehicle_type       252490 non-null  object
 2   registration_year  252490 non-null  int64 
 3   gearbox            252490 non-null  object
 4   power              252490 non-null  int64 
 5   model              252490 non-null  object
 6   kilometer          252490 non-null  int64 
 7   fuel_type          252490 non-null  object
 8   brand              252490 non-null  object
 9   repaired           252490 non-null  object
dtypes: int64(4), object(6)
memory usage: 21.2+ MB


<b> Промежуточный вывод: </b>
1. Данные загружены и изучены. Входные данные имели 354 369 строк и 16 столбцов.
2. Удалены следующие неинформативные признаки:
    * date_crawled — дата скачивания анкеты из базы;
    * registration_month — месяц регистрации автомобиля;
    * date_created — дата создания анкеты;
    * number_of_pictures — количество фотографий автомобиля;
    * postal_code — почтовый индекс владельца анкеты (пользователя);
    * last_seen — дата последней активности пользователя.
3. Удалены явные дубликаты.
4. Заполнены пропущенные значения в следующих стобцах:
    * vehicle_type;
    * gearbox;
    * model;
    * fuel_type;
    * repaired.
5. Обработаны аномалии в следующих столбцах:
    * price;
    * registration_year;
    * power.
   
После обработки данных получился датасет на 312 089 строк и 11 столбцов.

## Обучение моделей

### Подготовка выборок для обучения моделей

Разделим выборки на обучающую и тестовую (80/20)

In [32]:
features = data.drop('price', axis=1)
target = data['price']

features_train, features_test, target_train, target_test = train_test_split(features,
                                                                            target,
                                                                            test_size=0.2,
                                                                            random_state=STATE)
features_train.shape, target_train.shape,  features_test.shape, target_test.shape

((201992, 9), (201992,), (50498, 9), (50498,))

Выберем категориальные признаки

In [33]:
categorical_features = features_train.select_dtypes(include = 'object').columns.to_list()

In [34]:
categorical_features

['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired']

Выберем численные признаки

In [35]:
numeric_features = features_train.select_dtypes(include = 'number').columns.to_list()

In [36]:
numeric_features

['registration_year', 'power', 'kilometer']

<b> OneHotEncoder </b>

Кодируем и масштабируем признаки features_train

In [37]:
ohe_features_train = features_train.copy()
ohe_features_test = features_test.copy()

In [38]:
encoder_ohe = OneHotEncoder(drop = 'first', handle_unknown = 'ignore', sparse = False)
encoder_ohe.fit(ohe_features_train[categorical_features])
ohe_features_train[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(ohe_features_train[categorical_features])
ohe_features_train = ohe_features_train.drop(categorical_features, axis=1)

scaler = StandardScaler()
ohe_features_train[numeric_features] = scaler.fit_transform(ohe_features_train[numeric_features])
ohe_features_train.head()

,registration_year,power,kilometer,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,vehicle_type_suv,vehicle_type_wagon,...,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,repaired_unknown,repaired_yes
120988,0.463138,-0.413703,0.605057,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
232390,-0.162076,0.152078,0.605057,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
303437,-2.350324,-0.960626,-1.007344,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
283989,0.619441,0.152078,0.605057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93491,0.306834,-0.809751,0.605057,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Кодируем и масштабируем признаки features_test

In [39]:
ohe_features_test[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(ohe_features_test[categorical_features])
ohe_features_test = ohe_features_test.drop(categorical_features, axis=1)
ohe_features_test[numeric_features] = scaler.transform(ohe_features_test[numeric_features])
ohe_features_test.head()

,registration_year,power,kilometer,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,vehicle_type_suv,vehicle_type_wagon,...,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,repaired_unknown,repaired_yes
212963,0.463138,4.263425,0.605057,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194021,-0.005773,1.151626,0.605057,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
234617,0.775744,-0.262828,0.605057,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
250706,1.400958,-0.432563,-0.066777,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
258081,-0.318379,-0.621157,0.605057,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<b> OrdinalEncoder </b>

In [40]:
oe_features_train = features_train.copy()
oe_features_test = features_test.copy()

Кодируем и масштабируем признаки oe_features_train

In [41]:
encoder_oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

encoder_oe.fit(oe_features_train[categorical_features])
oe_features_train[
    encoder_oe.get_feature_names_out()
] = encoder_oe.transform(oe_features_train[categorical_features])

scaler = StandardScaler()
oe_features_train[numeric_features] = scaler.fit_transform(oe_features_train[numeric_features])

oe_features_train.head()

,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
120988,5.0,0.463138,1.0,-0.413703,79.0,0.605057,2.0,24.0,0.0
232390,7.0,-0.162076,1.0,0.152078,29.0,0.605057,2.0,1.0,0.0
303437,2.0,-2.350324,1.0,-0.960626,227.0,-1.007344,6.0,38.0,0.0
283989,0.0,0.619441,1.0,0.152078,222.0,0.605057,2.0,10.0,0.0
93491,5.0,0.306834,0.0,-0.809751,102.0,0.605057,6.0,10.0,2.0


Кодируем и масштабируем признаки oe_features_test

In [42]:
oe_features_test[
    encoder_oe.get_feature_names_out()
] = encoder_oe.transform(oe_features_test[categorical_features])
oe_features_test[numeric_features] = scaler.transform(oe_features_test[numeric_features])

oe_features_test.head()

,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
212963,4.0,0.463138,0.0,4.263425,179.0,0.605057,6.0,1.0,0.0
194021,4.0,-0.005773,0.0,1.151626,241.0,0.605057,2.0,2.0,0.0
234617,7.0,0.775744,1.0,-0.262828,10.0,0.605057,2.0,25.0,0.0
250706,7.0,1.400958,1.0,-0.432563,222.0,-0.066777,2.0,10.0,0.0
258081,7.0,-0.318379,1.0,-0.621157,10.0,0.605057,2.0,25.0,0.0


### Модель линейной регрессии

In [43]:
model_lr = LinearRegression()

In [44]:
# Создаем объект GridSearchCV для подбора гиперпараметров
grid_cv = GridSearchCV(estimator = model_lr, param_grid = {}, cv = 3, scoring = 'neg_root_mean_squared_error')

# Обучаем модель с подбором гиперпараметров
grid_cv.fit(ohe_features_train, target_train)

# Получаем лучшие гиперпараметры
best_params = grid_cv.best_params_

start_time = time.time()

# Создаем новую модель с лучшими гиперпараметрами
best_model_lr = LinearRegression(**best_params)

# Обучаем модель с лучшими гиперпараметрами
best_model_lr.fit(ohe_features_train, target_train)

end_time = time.time()
training_time_lr = end_time - start_time
print("Время обучения LinearRegression: %s секунд" % training_time_lr)

start_time = time.time()

#Предсказание цен на обучающей выборке
train_predictions_lr = best_model_lr.predict(ohe_features_train)

end_time = time.time()
prediction_time_lr = end_time - start_time
print("Время предсказания LinearRegression: %s секунд" % prediction_time_lr)

# Вычисление RMSE
rmse_lr = grid_cv.best_score_ * -1
print("RMSE LinearRegression: %.2f" % rmse_lr)

Время обучения LinearRegression: 6.672187805175781 секунд
Время предсказания LinearRegression: 0.3887350559234619 секунд
RMSE LinearRegression: 2604.91


### Модель LightGBM

In [45]:
start_time = time.time()

model_lgbm = LGBMRegressor(random_state=STATE, n_jobs=-1)

# Задаем сетку гиперпараметров для подбора
hyperparams = {
    'boosting_type': ['gbdt', 'dart'],
    'num_leaves': [10, 50, 100, 500, 1000, 5000],
    'learning_rate': [.1, .3, .5]
}

# Создаем объект GridSearchCV для подбора гиперпараметров
grid_cv = GridSearchCV(estimator = model_lgbm, param_grid = hyperparams, cv = 3, scoring = 'neg_root_mean_squared_error')

# Обучаем модель с подбором гиперпараметров
grid_cv.fit(oe_features_train, target_train)

# Получаем лучшие гиперпараметры
best_params = grid_cv.best_params_

end_time = time.time()
selection_time_lgbm = end_time - start_time
print("Время подбора гиперпараметров LightGBM: %s секунд" % selection_time_lgbm)

start_time = time.time()

# Создаем новую модель с лучшими гиперпараметрами
best_model_lgbm = LGBMRegressor(**best_params)

# Обучаем модель с лучшими гиперпараметрами
best_model_lgbm.fit(oe_features_train, target_train)

end_time = time.time()
training_time_lgbm = end_time - start_time

# Время обучения модели LightGBM
print("Время обучения LightGBM: %s секунд" % training_time_lgbm)

start_time = time.time()

# Предсказание цен на обучающей выборке
train_predictions_lgbm = best_model_lgbm.predict(oe_features_train)

end_time = time.time()
prediction_time_lgbm = end_time - start_time
print("Время предсказания LightGBM: %s секунд" % prediction_time_lgbm)

# Вычисление RMSE на валидационной выборке
rmse_lgb = grid_cv.best_score_ * -1
print("Лучшие гиперпараметры модели LightGBM: ", best_params)
print("RMSE LightGBM на валидационной выборке: %.2f" % rmse_lgb)

Время подбора гиперпараметров LightGBM: 6.002832889556885 секунд
Время обучения LightGBM: 1.8206830024719238 секунд
Время предсказания LightGBM: 1.0960278511047363 секунд
Лучшие гиперпараметры модели LightGBM:  {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'num_leaves': 500}
RMSE LightGBM на валидационной выборке: 1609.56


### Модель RandomForestRegressor

In [46]:
start_time = time.time()

model_rf = RandomForestRegressor(random_state=STATE, n_jobs=-1)

# Задаем сетку гиперпараметров для подбора
hyperparams = {
    'n_estimators': range(50, 251, 50),
    'max_depth': range(2, 15)
}

# Создаем объект GridSearchCV для подбора гиперпараметров
grid_cv = GridSearchCV(estimator = model_rf, param_grid = hyperparams, cv = 3, scoring = 'neg_root_mean_squared_error')

# Обучаем модель с подбором гиперпараметров
grid_cv.fit(oe_features_train, target_train)

# Получаем лучшие гиперпараметры
best_params = grid_cv.best_params_

end_time = time.time()
selection_time_rf = end_time - start_time
print("Время подбора гиперпараметров RandomForestRegressor: %s секунд" % selection_time_rf)

start_time = time.time()

# Создаем новую модель с лучшими гиперпараметрами
best_model_rf = RandomForestRegressor(**best_params)

# Обучаем модель с лучшими гиперпараметрами
best_model_rf.fit(oe_features_train, target_train)

end_time = time.time()
training_time_rf = end_time - start_time
print("Время обучения RandomForestRegressor: %s секунд" % training_time_rf)

start_time = time.time()

# Предсказание цен на обучающей выборке
train_predictions_rf = best_model_rf.predict(oe_features_train)

end_time = time.time()
prediction_time_rf = end_time - start_time
print("Время предсказания RandomForestRegressor: %s секунд" % prediction_time_rf)

# Вычисление RMSE на валидационной выборке
rmse_rf = grid_cv.best_score_ * -1

print("Лучшие гиперпараметры модели RandomForestRegressor: ", best_params)
print("RMSE RandomForestRegressor: %.2f" % rmse_rf)

Время подбора гиперпараметров RandomForestRegressor: 98.59464120864868 секунд
Время обучения RandomForestRegressor: 81.15410590171814 секунд
Время предсказания RandomForestRegressor: 6.715637922286987 секунд
Лучшие гиперпараметры модели RandomForestRegressor:  {'max_depth': 14, 'n_estimators': 250}
RMSE RandomForestRegressor: 1721.62


<b> Промежуточный вывод: </b>
1. Данные были разделены на обучающую и тестовую (80/20) выборки для обучения моделей.
2. Подготовлены ohe_features_train и ohe_features_test выборки путём кодирования категориальных признаков с использованием OneHotEncoder и масштабирования численных признков с использованием StandardScaler.
3. Подготовлены oe_features_train и oe_features_test выборки путём кодирования категориальных признаков с использованием OrdinalEncoder и масштабирования численных признков с использованием StandardScaler.
4. Обучены следующие модели:
    * LinearRegression;
    * LightGBM;
    * RandomForestRegressor.
      
Для каждой модели подоброны лучшие гиперпараметры, посчитаны время обучения, время предсказания и качество моделей.


## Анализ моделей

In [48]:
pd.DataFrame({'Модель': ['Linear Regression', 'LightGBM', 'RandomForestRegressor'],
                        'Время подбора гипперпараметров (сек.)': ['-', selection_time_lgbm, selection_time_rf],
                        'Время обучения (сек.)': [training_time_lr, training_time_lgbm, training_time_rf],
                        'Время предсказания (сек.)': [prediction_time_lr, prediction_time_lgbm, prediction_time_rf],
                        'Качество модели (RMSE)': [rmse_lr, rmse_lgb, rmse_rf]})

,Модель,Время подбора гипперпараметров (сек.),Время обучения (сек.),Время предсказания (сек.),Качество модели (RMSE)
0,Linear Regression,-,6.672188,0.388735,2604.914955
1,LightGBM,6.002833,1.820683,1.096028,1609.556113
2,RandomForestRegressor,98.594641,81.154106,6.715638,1721.619251


Значение метрики RMSE в модели LinearRegression превышает 2500. Лучшее значение метрики RMSE = 1623.53 имеет модель LightGBM. Оценим ее качество на тестовой выборке.

In [49]:
rmse_test = np.sqrt(mean_squared_error(target_test, best_model_lgbm.predict(oe_features_test)))
print("RMSE LightGBM на тестовой выборке: %.2f" % rmse_test)

RMSE LightGBM на тестовой выборке: 1564.49


Качество предсказаний, полученное на тестовой выборке, удовлетворяет обозначенному условию.

Проверим лучшую модель на адекватность

In [50]:
dummy_reg = DummyRegressor(strategy='mean')
dummy_reg.fit(ohe_features_train, target_train)
target_pred = dummy_reg.predict(ohe_features_train)
rmse = np.sqrt(mean_squared_error(target_train, target_pred))
print("RMSE DummyRegressor:", rmse)

RMSE DummyRegressor: 4674.269681437609


Результат тестирования нашей модели на тествой выборке оказался в разы лучше, чем результат константной модели DummyRegressor.

<b> Вывод: </b>
1. Данные загружены и изучены.
    * Заполнены пропущенные значения в следующих стобцах:  
        + vehicle_type;
        + gearbox;
        + model;
        + fuel_type;
        + repaired.
    * Обработаны аномалии в следующих столбцах:
        + price;
        + registration_year;
        + power.
    * Удалены следующие неинформативные признаки:
        + date_crawled — дата скачивания анкеты из базы;
        + registration_month — месяц регистрации автомобиля;
        + date_created — дата создания анкеты;
        + number_of_pictures — количество фотографий автомобиля;
        + postal_code — почтовый индекс владельца анкеты (пользователя);
        + last_seen — дата последней активности пользователя.
2. Данные были разделены на обучающую и тестовую (80/20) выборки, также были подготовлены дополнительные обучающие выборки путём кодирования категориальных признаков с использованием OneHotEncoder, OrdinalEncoder и масштабирования численных признков с использованием StandardScaler.
3. Обучены следующие модели с подобранными гипперпараметрами:
    * LinearRegression;
    * LightGBM;
    * RandomForestRegressor.
4. Самое быстрое время подбора гипперпараметров и обучения у модели LightGBM, время предсказания быстрее у Linear Regression. RandomForestRegressor в разы больше тратит время по всем пунктам.

Опираясь на критерии заказчика по скорости и качеству, <b> лучшей моделью является LightGBM </b>:
* время подбора гипперпараметров (сек.): 375.73;
* время обучения (сек.): 1.98;
* время предсказания (сек.): 0.5;
* качество модели (RMSE): 1564.49